<a href="https://colab.research.google.com/github/pfmoro/VisualizacaoRefugiadosVenezuela/blob/main/Prototype_Democracy_Index_and_Refugees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px
import os
from scipy.stats import spearmanr, linregress
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller

In [2]:
# Remove todos os arquivos CSV antigos
# Lista os arquivos atuais no ambiente de trabalho do Colab
files_in_colab = os.listdir('/content')

for file_name in files_in_colab:
    if file_name.endswith('.csv'):
        os.remove(file_name)
        print(f'Arquivo {file_name} removido.')

In [3]:
uploaded = files.upload()

Saving Oil Refugees GDP.csv to Oil Refugees GDP.csv


In [4]:
file_name = 'Oil Refugees GDP.csv'

# Verifica se o arquivo foi carregado corretamente
if file_name in uploaded:
    # Lê o arquivo CSV usando a biblioteca Pandas
    df_oil_ref_gdp = pd.read_csv(file_name,sep=";",encoding='latin-1')
else:
    print(f"O arquivo '{file_name}' não foi encontrado.")

In [5]:
uploaded = files.upload()

Saving CPI and DI.csv to CPI and DI.csv


In [6]:
df_oil_ref_gdp = df_oil_ref_gdp.replace('#DIV/0!', pd.NaT if 'datetime' in df_oil_ref_gdp.columns else pd.NA)

In [7]:
df_oil_ref_gdp.head(5)

,Year,Refugees Venezuela,Refugees Venezuela Variation,refugees normalized,Refugees Venezuela Variation %,Crude Oil Year Average,Crude Oil Year Average Variation %,Crude Oil Year Average Variation % Lag +,Crude Oil Year Average Variation % Lag 2+,Crude Oil Year Average Variation % Lag 3+,Crude Oil Year Average Variation % Lag -,GDP,GDP Variation
0,2001.0,798.0,NaN,NaN,NaN,24.351825,-0.137,NaN,NaN,NaN,0.0237,300198.0,NaN
1,2002.0,1304.0,506.0,0.000168,0.63,24.927748,0.024,-0.1374,NaN,NaN,0.1593,277865.0,-0.0744
2,2003.0,2614.0,1310.0,0.000435,1.00,28.898903,0.159,0.0237,-0.1374,NaN,0.3057,261376.0,-0.0593
3,2004.0,3933.0,1319.0,0.000438,0.50,37.733388,0.306,0.1593,0.0237,-0.1374,0.4150,317486.0,0.2147
4,2005.0,5397.0,1464.0,0.000486,0.37,53.391025,0.415,0.3057,0.1593,0.0237,0.2041,361227.0,0.1378


In [8]:
df_oil_ref_gdp.columns

Index(['Year', 'Refugees Venezuela', 'Refugees Venezuela Variation ',
       'refugees normalized', 'Refugees Venezuela Variation %',
       'Crude Oil Year Average', 'Crude Oil Year Average Variation %',
       'Crude Oil Year Average Variation % Lag +',
       'Crude Oil Year Average Variation % Lag 2+',
       'Crude Oil Year Average Variation % Lag 3+',
       'Crude Oil Year Average Variation % Lag -', 'GDP', 'GDP Variation'],
      dtype='object')

In [11]:
file_name = 'CPI and DI.csv'

# Verifica se o arquivo foi carregado corretamente
if file_name in uploaded:
    # Lê o arquivo CSV usando a biblioteca Pandas
    df_democracy = pd.read_csv(file_name,sep=";",encoding='latin-1')
else:
    print(f"O arquivo '{file_name}' não foi encontrado.")

In [12]:
df_democracy = df_democracy.replace('#DIV/0!', pd.NaT if 'datetime' in df_democracy.columns else pd.NA)

In [13]:
df_democracy['Year']=df_democracy['Year'].astype(int)

In [14]:
df_democracy.head(5)

,Entity,Code,Year,Corruption Perception Index - Transparency International (2018),Democracy Index
0,Venezuela,VEN,1998,23.0,NaN
1,Venezuela,VEN,1999,26.0,NaN
2,Venezuela,VEN,2000,27.0,NaN
3,Venezuela,VEN,2001,28.0,NaN
4,Venezuela,VEN,2002,25.0,NaN


In [16]:
df_democracy.columns

Index(['Entity', 'Code', 'Year',
       'Corruption Perception Index - Transparency International (2018)',
       'Democracy Index'],
      dtype='object')

In [18]:
df_democracy.columns=['Entity', 'Code', 'Year',
       'Corruption Perception Index',
       'Democracy Index']

In [19]:
df_democracy.columns

Index(['Entity', 'Code', 'Year', 'Corruption Perception Index',
       'Democracy Index'],
      dtype='object')

Gráfico simples - Refugiados(Normalizado) x Ano

In [17]:
fig = go.Figure([go.Scatter(x=df_oil_ref_gdp['Year'], y=df_oil_ref_gdp['refugees normalized'])])
fig.show()

In [ ]:
def rolling_spearman_slow(seqa, seqb, window):
    stridea = seqa.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    corrs = [spearmanr(a, b)[0] for (a,b) in zip(ssa, ssb)]
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)


In [21]:
#Remove NaN para plot:

var_x='Year'
var_y='Refugees Venezuela'
window=5

df_plot=df_oil_ref_gdp[[var_x, var_y]].dropna(how='any')
df_plot[var_x]=df_plot[var_x].astype(float)
df_plot[var_y]=df_plot[var_y].astype(float)*1e6


#Usaremos subplots:
fig = make_subplots(rows=3, cols=1)

#Cria um gráfico de dispersão para var_x
trace1 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot[var_y].astype(float),
    mode='lines+markers',
    name=var_x
)

# Gráfico de Exportação venezuela
trace2 = go.Scatter(
    x=df_democracy['Year'],
    y=df_democracy['Corruption Perception Index'],
    mode='lines+markers',
    name='Corruption Perception Index',
)

trace3 = go.Scatter(
    x=df_democracy['Year'],
    y=df_democracy['Democracy Index'],
    mode='lines+markers',
    name='Democracy Index',
)

fig.update_layout(height=600, width=800,
    barmode='stack',
    title='Numero de Refugiados e indices de segurança institucional',
)

# Define os dados que serão plotados
data = [trace1, trace2, trace3]
fig.add_trace(trace1,row=1, col=1)
fig.add_trace(trace2,row=2, col=1)
fig.add_trace(trace3,row=3, col=1)


# Plota o gráfico no Google Colab
#plot(fig)
fig.show()
